### Faiss

Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains supporting code for evaluation and parameter tuning.

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [5]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [7]:
embeddings=OllamaEmbeddings(model="mxbai-embed-large")
db=FAISS.from_documents(docs,embeddings)
db

In [8]:
### querying

query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

### As a Retriever

We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

This retrievers are like an interface, like when ever we put any kind of query, it can be connected to the vector store db only when we connect this vector store db into retriever class. So it acts like an interface which will be able to retrieve the details from the vector store and provide you response.

When we work with different LLM models, at that point of time, I cannot directly use vector store DB, but initially I need to convert this vector store db into a retriever and then finally use it with any LLM models.

In [9]:
retriever=db.as_retriever()
retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score

There are some FAISS specific methods. One of them is similarity_searcch_with_score, which allows you to return not only the documents but also the distance score of the query of them. The returned distance score is L2 distance(Manhattan distance). Therefore, a lower score is better.

In [10]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='1982b49c-137d-4519-9a03-2f6fbc8d5e2a', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(204.05083)),
 (Document(id='fa1efbb8-9de7-415d-87d9-0bd8bb9a7e76', metadata={'source': 'spe

We can also pass vectors instead of sentences like below

In [12]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[0.5115804672241211,
 -0.3932502865791321,
 -0.28175410628318787,
 0.44659358263015747,
 -0.7385181784629822,
 -1.5118913650512695,
 -0.7009395956993103,
 0.3404998183250427,
 0.04491307586431503,
 -0.13477295637130737,
 0.28562188148498535,
 0.05327965319156647,
 -0.3357430696487427,
 0.5870353579521179,
 -0.3931404650211334,
 -0.4126819372177124,
 -0.7173913717269897,
 -0.20877325534820557,
 0.25617098808288574,
 0.0503106527030468,
 -0.03321405500173569,
 -0.4084383249282837,
 -0.6080053448677063,
 0.38306528329849243,
 -0.11254997551441193,
 0.2734755277633667,
 -0.6003841161727905,
 0.20301209390163422,
 1.1313536167144775,
 -0.11163868010044098,
 -0.17414331436157227,
 0.5402445197105408,
 -0.04121880978345871,
 -0.687071681022644,
 0.4343479573726654,
 -0.6323160529136658,
 0.42785125970840454,
 -1.9080837965011597,
 0.1345849633216858,
 -0.15895980596542358,
 0.2525251507759094,
 0.2905258536338806,
 0.5584207773208618,
 -0.4569806456565857,
 -1.4715574979782104,
 -0.2829653024

In [14]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='1982b49c-137d-4519-9a03-2f6fbc8d5e2a', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='fa1efbb8-9de7-415d-87d9-0bd8bb9a7e76', metadata={'source': 'speech.txt'}, page_content='…\n

In [15]:
### Saving and loading 
db.save_local("faiss_index")

In [17]:
## Load this folder
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)

#allow_dangerous_deserialization=True says you can trust this file to load
docs=new_db.similarity_search(query)

In [19]:
docs

[Document(id='1982b49c-137d-4519-9a03-2f6fbc8d5e2a', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='fa1efbb8-9de7-415d-87d9-0bd8bb9a7e76', metadata={'source': 'speech.txt'}, page_content='…\n

In [23]:
new_db